In [1]:
# The Main Function for APAD Project -- Cloth On Fly

# Author: Tianyi (Kelly) Zhang
#         Shikha Singh

# Creation Date: 07/21/2019

# Major: MSITM

In [2]:
import sqlite3
from uuid import uuid4
from datetime import date
from datetime import date, datetime, timedelta
import csv
import glob

### Databate Create

In [3]:
db = sqlite3.connect("ClothONFly.db")
cursor = db.cursor()

In [4]:
# cursor.execute('''
#               CREATE TABLE Users(User_ID INTEGER PRIMARY KEY AutoIncrement, User_Name Text NOT NULL UNIQUE, Password Text,
#               User_Type Text Not Null check(User_Type = "Admin" or User_Type = "Buyer" or User_Type = "Seller"), First_Name TEXT NOT NULL,
#               Last_Name TEXT NOT NULL, Email TEXT Not NUll, Address Text Not NUll, Phone_Num INTEGER Not Null)
#             ''')

# cursor.execute('''
#               CREATE TABLE Inventory_Items(Item_ID INTEGER PRIMARY KEY AutoIncrement, Brand_Name TEXT NOT NULL,
#               Cloth_Type TEXT NOT NULL, Size TEXT NOT NULL, Gender TEXT NOT NULL, Original_Price REAL NOT NULL,
#               Rental_Price REAL NOT NULL, Available_From DATE, Owner_ID INTEGER, Location TEXT NOT NULL, Cloth_Image BLOB NOT NULL,
#               Foreign Key(Owner_ID) References Users(User_ID))
#             ''')

# cursor.execute('''
#               CREATE TABLE Orders (Order_ID INTEGER PRIMARY KEY AutoIncrement, User_ID INTEGER NOT NULL, Item_ID INTEGER Not Null,
#               Delivery_Date Date Not Null, Return_Date Date Not Null, Shipping_Address Text Not Null,
#               Order_Status Text Not Null check(Order_Status = "Order Received(Shipment Pending)" or Order_Status = "Order Shipped" or Order_Status = "Return Initiated" or Order_Status = "Return Received(Deposit Refunded)" or Order_Status = "Return Not Received(Deposit Withheld)"),
#               Shipping_Date Date, FOREIGN KEY(User_ID) REFERENCES Users(User_ID), FOREIGN KEY(Item_ID) References Inventory_Item(Item_ID))
#           ''')

In [5]:
# cursor.execute('''drop table Users''')
# cursor.execute('''drop table Inventory_Items''')
# cursor.execute('''drop table Orders''')

In [6]:
db.commit()
cursor.close()
db.close()

### utils file

In [7]:
ADMIN_ROLE_NAME = "Admin"
BUYER_ROLE_NAME = "Buyer"
SELLER_ROLE_NAME = "Seller"

ORDER_RECEIVED = "Order Received(Shipment Pending)"
ORDER_RETURN_INIT = "Return Initiated"

TEST_ADMIN_NAME = 'KellyZhang'
TEST_PASSWORD = '123'
TEST_FIRST_NAME = 'Kelly'
TEST_LAST_NAME = 'Zhang'
TEST_EMAIL = '123abc@def.com'
TEST_ADDRESS = '123 FASDF, AUSTIN, TX 78731'
TEST_PHONE_NUM = 1234567890
TEST_ID_NUM = 0

### activities.py

In [8]:
def register(user_name, password, user_role, first_name, last_name, email, address, phone_num, id_num) -> bool:

    if user_role.lower() == ADMIN_ROLE_NAME.lower():
        new_admin_user = Admin(user_name, password, first_name,
                               last_name, email, address, phone_num, id_num)
        write_user_into_database(ADMIN_ROLE_NAME, new_admin_user)
        print("Register succeeded! Returning to the main menu...\n")
        
    elif user_role.lower() == SELLER_ROLE_NAME.lower():
        new_seller_user = Seller(user_name, password, first_name,
                               last_name, email, address, phone_num, id_num)
        move_seller_to_pending(new_seller_user)
        print("Your registration is pending for authorization, thank you for your patience.")
        
    elif user_role.lower() == BUYER_ROLE_NAME.lower():
        print('This user is a buyer.')
        new_buyer_user = Buyer(user_name, password, first_name,
                                 last_name, email, address, phone_num, id_num)
        print('Writing the new buyer into db...')
        write_user_into_database(BUYER_ROLE_NAME, new_buyer_user)
        print("Register succeeded! Returning to the main menu...\n")
        
    else:
        print("Unknown role. Please try again.\n")
        return False

    return True

In [9]:
def check_duplicate(user_name):
    db = sqlite3.connect("ClothONFly.db")
    cursor = db.cursor()
    try:
        cur_result = cursor.execute('''SELECT * FROM Users where User_Name = ? ''',(str(user_name),))
        db.commit()
        if len(cur_result.fetchall()) != 0:
            cursor.close()
            db.close()
            return True
        else:
            cursor.close()
            db.close()
            return False
    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        return True

In [10]:
def login(user_name, password):
    user_name = TEST_ADMIN_NAME
    password = TEST_PASSWORD
    try: 
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        cur_result = cursor.execute('''SELECT * FROM Users where User_Name = ? AND Password = ? ''',(user_name, password))
        db.commit()
        if len(cur_result.fetchall()) != 0:
            print("LogIn Succeed")
            cursor.close()
            db.close()
            return True
        else:
            print("Unable to LogIn")
            cursor.close()
            db.close()
            return False
        print("db is closed!")
        return True
    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        return False
    

In [11]:
def return_role(user_name):
    db = sqlite3.connect("ClothONFly.db")
    cursor = db.cursor()
    
    try:
        cur_result = cursor.execute('''SELECT User_Type FROM Users where User_Name = ? ''', (str(user_name),))
        cur_result_all = cur_result.fetchall()
        db.commit()
        cursor.close()
        db.close()
        
        for row in cur_result_all:
            return row[0]
        
    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        

In [12]:
def write_user_into_database(user_role, user) -> bool:
    try: 
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        cursor.execute('''Insert into Users(User_Name, Password, User_Type, First_Name, Last_Name, Email, Address, Phone_Num, User_ID) VALUES(?,?,?,?,?,?,?,?,?)''',
                       (user.user_name, user.password, user_role, user.first_name, user.last_name, user.email, user.address, user.phone_num, None))
        db.commit()
        cursor.close()
        db.close()
        print("db is closed!")
        return True
    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        return False

In [13]:
PENDING_SELLER_LIST = list()
def move_seller_to_pending(seller) -> bool:
    PENDING_SELLER_LIST.append(seller)
    return True

In [14]:
def get_userid_from_username(user_name):
    print("get_userid_by_username")
    try: 
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        cursor.execute('''SELECT User_ID FROM Users WHERE User_Name = ?''', (user_name,))
        db.commit()
        cursor.close()
        db.close()
        print("db is closed!")
        return True
    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        return False

### admin.py

In [15]:
class Admin:

    def __init__(self, user_name, password, first_name, last_name, email, address, phone_num, id_num):
        self.user_name = user_name
        self.password = password
        self.first_name = first_name
        self.last_name = last_name
        self.email = email
        self.address = address
        self.phone_num = phone_num
        self.id_num = id_num
    
    @staticmethod
    def authorize_sellers() -> bool:
        print("Hi Admin! Please check and authorize the following sellers for registration:")
        
        while (len(PENDING_SELLER_LIST) > 0):
            seller = PENDING_SELLER_LIST[0]
            print('PENDING_SELLER_LIST size: ' + str(len(PENDING_SELLER_LIST)) + "\n")
            # Notify the current admin about the seller's info
            print("Seller basic: \n", 
                  ((seller.user_name), (seller.password), (SELLER_ROLE_NAME), 
                   (seller.first_name), (seller.last_name), (seller.email), 
                   (seller.address), (seller.phone_num), (seller.id_num)))
            decision = input('Do you want to grant this seller permission to the system (Y/N)? ')
            if decision.lower() == 'y':
                print("Writing seller into database...")
                write_user_into_database(SELLER_ROLE_NAME, seller)
                print("Finished.\n")

            print("Removing the seller from the pending list...")
            PENDING_SELLER_LIST.remove(seller)
            print('PENDING_SELLER_LIST size: ' + str(len(PENDING_SELLER_LIST)) + "\n")
            
        print("Back to Admin Menu...\n")
        return True

    @staticmethod
    # Remove a user from the user table if needed
    def remove_user() -> bool:
        answer = input("Please input the user name you want to remove: ")
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''DELETE FROM Users 
                              WHERE User_Name = ?''',(answer,))
            db.commit()
            cursor.close()
            db.close()
            return True
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False

        return False

### seller.py

In [16]:
class Seller:
    image_dir = "Images/"
    export_dir = "Exports/"
    import_dir = "ImportFiles/"

    @staticmethod
# This function add the clothing items in the database
# The items are available for renting from the day they are added in the system. 
# So we have used date.today() function to populate Available_From unless specified by seller explicitly
    def add_item(Brand_Name,Type,Size,Gender,Original_Price,Rental_Price,Owner_ID,Location,Cloth_Image,Deposit,Available_From=None) :

        try:
            db = sqlite3.connect("ClothONFly.db")
            cursor = db.cursor()
        except Error as e:
            print(e)
            return None
        
        try:
#             Checking the images in the specide image directory
            for image in glob.glob(Seller.image_dir+'/*'): 
#             Checking if the image specified by the seller is available in the image directory
                if (str(image).replace(Seller.image_dir,'')) == Cloth_Image:
                    with open(image, 'rb') as file:
                        blobData = file.read()
        except Error as e:
            print("Could not read file. Check your file name and folder path")
            
        if Available_From == "" or Available_From is None:
            Available_From = date.today()
            
        cursor.execute(
            '''Insert into Inventory_Items(Brand_Name,Type,Size,Gender,Original_Price,Rental_Price,Owner_ID,Location,Cloth_Image,Deposit,Available_From) values(?,?,?,?,?,?,?,?,?,?,?)''',
            (Brand_Name,Type,Size,Gender,Original_Price,Rental_Price,Owner_ID,Location,blobData,Deposit,Available_From))
        db.commit()

        db.close()

    @staticmethod
    def remove_item(Item_ID):
#         This function removes the inventory item from the database
        try:
            db = sqlite3.connect("ClothONFly.db")
            cursor = db.cursor()
        except Error as e:
            print(e)
            return None
        cursor.execute('''Delete from Inventory_Items where Item_ID == ?''',(str(Item_ID)))
        db.commit()
        db.close()

    @staticmethod
    def update_item(Item_ID,Brand_Name,Type,Size,Gender,Original_Price,Rental_Price,Owner_ID,Location,Cloth_Image,Deposit,Available_From):
#        This function updates any/all attributes of the inventory item in the database
        try:
            db = sqlite3.connect("ClothONFly.db")
            cursor = db.cursor()
        except Error as e:
            print(e)
            return None
        
        try:
            for image in glob.glob(Seller.image_dir + '/*'):
                if (str(image).replace(Seller.image_dir, '')) == Cloth_Image:
                    with open(image, 'rb') as file:
                        blobData = file.read()
        except Error as e:
            print("Could not read file. Check your file name and folder path")
            
        cursor.execute('''Update Inventory_Items 
        set Brand_Name= ?,
        Type = ? ,
        Size =?,
        Gender = ?,
        Original_Price =?,
        Rental_Price =?,
        Owner_ID=?,
        Location=?,
        Cloth_Image=?,
        Deposit=?,
        Available_From=? where Item_ID = ?''',(Brand_Name,Type,Size,Gender,Original_Price,Rental_Price,Owner_ID,Location,blobData,Deposit,Available_From,Item_ID))
        db.commit()
        db.close()

    @staticmethod
    def ship_item(Order_ID):
# This function will be called when the Seller clicks on ship item button. 
# This will change the status of the order to "Order Shipped"
# This will also update the shipping date to current date

        try:
            db = sqlite3.connect("ClothONFly.db")
            cursor = db.cursor()
        except Error as e:
            print(e)
            return None
        cursor.execute('''Update Orders 
                set Order_Status = 'Order Shipped',
                    Shipping_Date = ?
                where Order_ID = ?''', (date.today(),Order_ID))
        db.commit()
        db.close()

    @staticmethod
    def refund_deposit(Order_ID):
# This function will be called when the Seller clicks on refund deposit button. 
# This function will update the order status to "Return Received(Deposit Refunded)" 
# This function will update the item's Available From date as current date
# This function will execute when the return has been initited by the buyer(order status is "return Initiated")
# and return date is current day or a future date


        try:
            db = sqlite3.connect("ClothONFly.db")
            cursor = db.cursor()
        except Error as e:
            print(e)
            return None
        cursor.execute('''Update Orders 
                    set Order_Status = 'Return Received(Deposit Refunded)'
                    where Order_ID = ? and Return_Date >= ? and Order_Status = "Return Initiated" ''', (str(Order_ID),date.today()))
        
        cursor.execute('''select Item_ID from Orders where Order_ID = ?''',(Order_ID,))
        for row in cursor.fetchall():
            item = row[0]
        cursor.execute('''Update Inventory_Items 
                    set Available_From =? where Item_ID = ?''', (date.today(),item))
        db.commit()
        db.close()
        
    @staticmethod
    def withhold_deposit(Order_ID):
# This function will be called when the Seller clicks on withhold deposit button. 
# This function will update the order status to "Return Not Received(Deposit Withheld)" 
# This function will execute when the return has NOT been initited by the buyer(order status is NOT "return Initiated")
# and return date is less that current date i.e. a past date
        try:
            db = sqlite3.connect("ClothONFly.db")
            cursor = db.cursor()
        except Error as e:
            print(e)
            return None
        cursor.execute('''Update Orders 
                        set Order_Status = 'Return Not Received(Deposit Withheld)'
                        where Order_ID = ? and Return_Date < ? and Order_Status != "Return Initiated" ''', (str(Order_ID),date.today()))
        db.commit()
        db.close()

    @staticmethod
    def bulk_item_upload(filename):
# This function add the clothing items in the database in bulk
# This function reads from the csv file provided by the Seller and adds each row of the csv as a new inventory item

#             Checking the file in the specide import file directory
        for files in glob.glob(Seller.import_dir + '/*'):
#             Checking if the file specified by the seller is available in the directory
            if (str(files).replace(Seller.import_dir, '')) == filename:
                with open(Seller.import_dir + '/'+filename, 'r') as inputfile:
                    items = csv.DictReader(inputfile)

                    for item in items:
                        if item['AvailableFrom'] == '':
                            item['AvailableFrom'] = date.today()
                        Seller().add_item(item['Brand'],item['Type'],item['Size'],item['Gender'],item['OriginalPrice'],item['RentalPrice'],item['OwnerID'],item['Location'],item['ClothImage'],item['Deposit'],item['AvailableFrom'])
    
    @staticmethod
    def all_orders():
# This function takes fetches all orders in the system along with the item, buyer and seller details
# This function creates a csv file in the specified directory and writes the fetch result in the file
# The file is created with a date time stamp so that every time this function is executed a new file is created
        try:
            db = sqlite3.connect("ClothONFly.db")
            cursor = db.cursor()
        except Error as e:
            print(e)
            return None
        cursor.execute('''select o.Order_ID,o.Order_Status As "Order Status", o.Delivery_Date As "Delivery Date",o.Return_Date As "Return Date",o.Shipping_Address AS "Shipping Address",
                                u.First_Name AS "Buyer's First Name",u.Last_Name AS "Buyer's LastName",u.Email AS "Buyer's Email",u.Phone_Num AS "Buyer's Contact",
                                i.Item_ID As "Product ID",i.Brand_Name AS "Product's Brand",i.Type AS "Product Type", i.Size AS "Product Size",i.Gender AS "Gender",
                                usr.First_Name as "Seller's first Name", usr.Last_Name as "Seller's Last Name", usr.Email As "Seller's Email",usr.Phone_Num AS "Seller's Contact"
                                from Orders o join Users u on u.User_ID=o.User_ID join Inventory_Items i on o.Item_ID = i.Item_ID join Users usr on i.Owner_ID =usr.User_ID ''')

        with open(Seller.export_dir+'/'+"Orders_export_"+ str(datetime.now()) + ".csv", "w", newline='') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerow([i[0] for i in cursor.description])  # write headers
            csv_writer.writerows(cursor)
        db.close()
        
    @staticmethod
    def orders_per_seller(Seller_ID):
# This function fetches the orders from a system along with the item and buyer for a specified seller
# This function creates a csv file in the specified directory and writes the fetch result in the file
# The file is created with the seller's username and date time stamp so that every time this function is executed a new file is created
        try:
            db = sqlite3.connect("ClothONFly.db")
            cursor = db.cursor()
        except Error as e:
            print(e)
            return None
        cursor.execute('''select User_Name from Users where User_ID = ?''',(Seller_ID,))
        for row in cursor.fetchall():
            seller = row[0]
        cursor.execute('''select o.Order_ID,o.Order_Status As "Order Status", o.Delivery_Date As "Delivery Date",o.Return_Date As "Return Date",o.Shipping_Address AS "Shipping Address",
                                        u.First_Name AS "Buyer's First Name",u.Last_Name AS "Buyer's LastName",u.Email AS "Buyer's Email",u.Phone_Num AS "Buyer's Contact",
                                        i.Item_ID As "Product ID",i.Brand_Name AS "Product's Brand",i.Type AS "Product Type", i.Size AS "Product Size",i.Gender AS "Gender"
                                        from Orders o join Users u on u.User_ID=o.User_ID join Inventory_Items i on o.Item_ID = i.Item_ID 
                                        where i.Owner_ID = ? ''', (Seller_ID,))
        
        for row in cursor.fetchall():
            print(row)
        
        print("\n \n An export of this is placed in the Exports directory with Seller's name and current datetimestamp")

        try:
            with open(Seller.export_dir + '/' + seller + "'s_Orders_export_" + str(datetime.now()) + ".csv", "w", newline='') as file:
                csv_writer = csv.writer(file)
                csv_writer.writerow([i[0] for i in cursor.description])  # write headers
                csv_writer.writerows(cursor)
        except:
            print("Could not write to the file")
        
        db.close()
        
    @staticmethod
    def buyer_order_history(User_ID):
# This function fetches the orders for a user
        try:
            db = sqlite3.connect("ClothONFly.db")
            cursor = db.cursor()
        except Error as e:
            print(e)
            return None
        cursor.execute('''select * from Orders where User_ID = ?''',(User_ID,))
        for row in cursor.fetchall():
            print(row)
        db.close()

# buyer.py

In [17]:
class Buyer:

    def __init__(self, user_name, password, first_name, last_name, email, address, phone_num, id_num):
        self.user_name = user_name
        self.password = password
        self.first_name = first_name
        self.last_name = last_name
        self.email = email
        self.address = address
        self.phone_num = phone_num
        self.id_num = id_num

    @staticmethod    
    def view_item():
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''SELECT * FROM Inventory_Items''')
            result = cursor.fetchall()

            for item in result:
                print("Inventory size: " + str(len(result)))
                print("Item_ID: " + str(item[0]) + "; Brand_Name: " + str(item[1]) + "; Cloth_Type: " + str(item[2]) + 
                      "; Size: " + str(item[3]) + "; Gender: " + str(item[4]) + "; Original_Price: " + str(item[5]) + 
                      "; Rental_Price: " + str(item[6]) + "; Available_From: " + str(item[7]) + 
                      "; Owner_ID: " + str(item[8]) + "; Location: " + str(item[9]) + "; Cloth_Image: " + str(item[10]) + ".\n")
            print("\n")
            
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return
        
        cursor.close()
        db.close()
    
    @staticmethod
    def rent_item(id_num) -> bool:
        print("Renting item")
        item_id_num = input("Please input the item num you want to rent: ")
        shipping_address = input("Please input your shipping address: ")
        rent_days = int(input("Please input how many days you want to rent the item: "))
        delivery_date = date.today() + timedelta(days=7)
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''INSERT INTO Orders(Order_ID, User_ID, Item_ID, Delivery_Date, Return_Date, Shipping_Address, Order_Status, Shipping_Date)
                              VALUES(?,?,?,?,?,?,?,?)
                           ''',(None, id_num, item_id_num, delivery_date, delivery_date + timedelta(days = rent_days), shipping_address, ORDER_RECEIVED, ""))
#             # update Available_From in Inventory_Items table 
#             cursor.execute('''UPDATE Inventory_Items
#                               WHERE Available_From = ? ''', (shipping_date + timedelta(delivery_date) + timedelta(days = rent_days)))
            db.commit()
            cursor.close()
            db.close()
            return True
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False
    
    @staticmethod
    def return_item() -> bool:
        print("Returning item")
        item_id_num = input("Please input the item num you want to return: ")
        # update Order_Status in Orders table
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''UPDATE Orders SET Order_Status = ? WHERE Item_ID = ?''', (ORDER_RETURN_INIT, item_id_num,))
            db.commit()
            cursor.close()
            db.close()
            return True
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False
        return False
    

### main.py

In [18]:
def show_main_menu() -> bool:
    print("Hi there! Welcome to Cloth on Fly! What's Next?")
    print("[1] Register")
    print("[2] Log In")
    print("[3] Quit")
    main_answer = input("Input the action number here: ")
    # registration
    if main_answer == "1":
        print("Please input the following information to register!")
        
        user_name = input("Username: ")
        
        is_user_name_duplicate = check_duplicate(user_name)
        if (is_user_name_duplicate):
            print("This user name already exists in the database. Please use a different one for registration\n")
            return True
        
        password = input("Password: ")
        user_role = input("Do you want to register as admin/seller/buyer: ")
        first_name = input("First Name: ")
        last_name = input("Last Name: ")
        email = input("Email: ")
        address = input("Address: ")
        phone_num = int(input("Phone Number: "))
        
        register(user_name, password, user_role, first_name, last_name, email, address, phone_num)
        return True

    # login
    elif main_answer == "2":
        print("Hey Flyer! Please input your user name and password to log in: ")
        user_name = input("Username: ")
        password = input("Password: ")
        user_role = return_role(user_name)
        if user_role == None:
            print("User does not exist. Please try again.\n")
        else:
            print("User role is: " + user_role + "\n")
            user_menu_keep_looping = True
            while (user_menu_keep_looping):
                user_menu_keep_looping = show_user_menu(user_name, user_role)
        return True
    
    # quit
    elif main_answer == "3":
        print("Thanks for using Cloth ON Fly!\n")
        return False
    
    # input other options
    else:
        print("Input Unrecognized. Please try again!\n")
        return True

In [19]:
def show_user_menu(user_name, user_role):
    if (user_role == ADMIN_ROLE_NAME):
        return show_admin_menu()
    elif (user_role == SELLER_ROLE_NAME):
        return show_seller_menu()
    else:
        return show_buyer_menu(user_name)

In [20]:
def show_admin_menu():
    print(">>>>>>>> Admin Menu\n")
    print("[1] Authorize Sellers")
    print("[2] Remove An User")
    print("[3] Add An Item")
    print("[4] Remove An Item")
    print("[5] Update An Item")
    print("[6] Check Orders")
    print("[7] Log Out")
    user_answer = input("Input the action number here: ")
    
    if user_answer == '1':
        print("Retrieving pending sellers list...\n")
        Admin.authorize_sellers()
        print("Finished pending sellers review.\n")
        return True
    elif user_answer == '2':
        print("Removing an user...\n")
        Admin.remove_user()
        print("Finished user deletion.\n")
        return True
    elif user_answer == '3':
        print("Adding an item...\n")
        return True
    elif user_answer == '4':
        print("Removing an item...\n")
        return True
    elif user_answer == '5':
        print("Updating an item...\n")
        return True
    elif user_answer == '6':
        print("Checking all existing orders...\n")
        return True
    elif user_answer == '7':
        print("Thanks for using Cloth ON Fly!\n")
        return False
    
    # input other options
    else:
        print("Input Unrecognized. Please try again!\n")
        return True

In [21]:
def show_seller_menu():
    print(">>>>>>>> Seller Menu\n")
    print("[1] Add An Item")
    print("[2] Remove An Item")
    print("[3] Update An Item")
    print("[4] Add items in bulk")
    print("[5] Show orders for me")
    print("[6] Ship An Item")
    print("[7] Refund Deposit")
    print("[8] Withhold Deposit")
    print("[9] Log Out")
    user_answer = input("Input the action number here: ")
    
    if user_answer == '1':
        Brand_Name = input("Enter the brand name: ")
        Type = input("Enter the Type of clothing item: ")
        Size = input("Enter the Size of the clothing item: ")
        Gender = input("Enter the Gender for whom the item should be available: ")
        Original_Price = input("Enter the Original Price of the item(USD): ")
        Rental_Price = input("Enter the Rental Price per day (USD): ")
        Owner_ID = input("Enter the Seller's ID to be associated with the item: ")
        Location = input("Enter the Location where the item is available: ")
        Cloth_Image = input("Enter the Image file name. Please ensure the image is placed in the images folder: ")
        Deposit = input("Enter the Deposit amount: ")
        Available_From = input("Enter the Date from when the item is avialable for rent: ")
        print("Adding an item...\n")
        Seller.add_item(Brand_Name,Type,Size,Gender,Original_Price,Rental_Price,Owner_ID,Location,Cloth_Image,Deposit,Available_From)
        print("Item added to Inventory ...\n")
        return True
    elif user_answer == '2':
        print("Removing an item...\n")
        Item_ID = input("Enter the Item ID that you want to remoive from the inventory:")
        Seller.remove_item(Item_ID)
        print("Item removed from Inventory ...\n")
        return True
    elif user_answer == '3':
        Item_ID = input("Enter the Item ID that you want to update:")
        Brand_Name = input("Enter the brand name: ")
        Type = input("Enter the Type of clothing item: ")
        Size = input("Enter the Size of the clothing item: ")
        Gender = input("Enter the Gender for whom the item should be available: ")
        Original_Price = input("Enter the Original Price of the item(USD): ")
        Rental_Price = input("Enter the Rental Price per day (USD): ")
        Owner_ID = input("Enter the Seller's ID to be associated with the item: ")
        Location = input("Enter the Location where the item is available: ")
        Cloth_Image = input("Enter the Image file name. Please ensure the image is placed in the images folder: ")
        Deposit = input("Enter the Deposit amount: ")
        Available_From = input("Enter the Date from when the item is avialable for rent: ")
        print("Updating an item...\n")
        Seller.update_item(Item_ID,Brand_Name,Type,Size,Gender,Original_Price,Rental_Price,Owner_ID,Location,Cloth_Image,Deposit,Available_From)
        print("Item Updated...\n")
        return True
    elif user_answer == '4':
        print("Enter all the item details in a csv file and place it in ImportFiles directory...\n")
        filename = input("Enter the name of the file to be loaded: ")
        print("Loading items...\n")
        Seller.bulk_item_upload(filename)
        return True
    elif user_answer == '5':
        Seller_ID = input("Enter your Seller ID:")
        print("Displaying all orders for you...\n")
        Seller.orders_per_seller(Seller_ID)
        return True
    elif user_answer == '6':
        Order_ID = input("Enter the Order_ID that you want to ship:")
        print("Shipping your order...\n ")
        Seller.ship_item(Order_ID)
        print("Order shipped...\n")
        return True
    elif user_answer == '7':
        Order_ID = input("Enter the Order_ID that you want to refund:")
        print("Refunding deposit for the order...\n ")
        Seller.refund_deposit(Order_ID)
        print("Deposit refunded...\n")
        return True
    elif user_answer == '8':
        Order_ID = input("Enter the Order_ID that you want to withhold:")
        print("Withholding deposit for the order...\n ")
        Seller.withhold_deposit(Order_ID)
        print("Deposit withheld...\n")
        return True
    
    # input other options
    else:
        print("Input Unrecognized. Please try again!\n")
        return True

In [22]:
def show_buyer_menu(user_name):
    print(">>>>>>>> Buyer Menu\n")
    print("[1] View Items")
    print("[2] Rent An Item")
    print("[3] Return An Item")
    print("[4] Log Out")
    user_answer = input("Input the action number here: ")
    
    user_id = get_userid_from_username(user_name)
    
    if user_answer == '1':
        print("Checking inventory...\n")
        Buyer.view_item()
        return True
    elif user_answer == '2':
        print("Placing an order...\n")
        Buyer.rent_item(user_id)
        print("Complete the order!\n")
        return True
    elif user_answer == '3':
        print("Item return processing...\n")
        Buyer.return_item()
        print("Return Initiated\n")
        return True
    elif user_answer == '4':
        print("Thanks for using Cloth ON Fly!\n")
        return False
    
    # input other options
    else:
        print("Input Unrecognized. Please try again!\n")
        return True
    

In [ ]:
def main():
    keep_looping = True
    while (keep_looping):
        keep_looping = show_main_menu()
    
if __name__ == '__main__':
    main()

Hi there! Welcome to Cloth on Fly! What's Next?
[1] Register
[2] Log In
[3] Quit
Input the action number here: 2
Hey Flyer! Please input your user name and password to log in: 
Username: Seller
Password: Seller@1
User does not exist. Please try again.

Hi there! Welcome to Cloth on Fly! What's Next?
[1] Register
[2] Log In
[3] Quit
Input the action number here: 2
Hey Flyer! Please input your user name and password to log in: 
Username: Seller1
Password: Seller@1
User role is: Seller

>>>>>>>> Seller Menu

[1] Add An Item
[2] Remove An Item
[3] Update An Item
[4] Add items in bulk
[5] Show orders for me
[6] Ship An Item
[7] Refund Deposit
[8] Withhold Deposit
[9] Log Out
Input the action number here: 1
Enter the brand name: ZZZ
Enter the Type of clothing item: ZZZ
Enter the Size of the clothing item: M
Enter the Gender for whom the item should be available: Male
Enter the Original Price of the item(USD): 100
Enter the Rental Price per day (USD): 10
Enter the Seller's ID to be associated